In [ ]:
import os
%pwd
# os.chdir('../')
%pwd

'c:\\Projects\\FinSight'

In [ ]:
#config/config.yaml
'''
artifacts_root: artifacts

data_ingestion:
  root_dir: artifacts/data_ingestion
  raw_data_file: raw_data.csv
'''

In [ ]:
# src/finance_ml/entity/config_entity.py

from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: __path__
    raw_data_file: __path__
    period: str
    interval: str

In [3]:
# src/finance_ml/config/configuration.py

from finance_ml.constants import *
from finance_ml.utils.common import read_yaml, create_directories
from finance_ml.entity.config_entity import DataIngestionConfig
import os

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        params = self.params.data_ingestion # Access data_ingestion params from params.yaml

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            raw_data_file=os.path.join(config.root_dir, config.raw_data_file), 
            period=params.period,     
            interval=params.interval  
        )

        return data_ingestion_config

In [ ]:
import yfinance as yf

class DataIngestion:
    def __init__(self, config, ticker):
        self.config = config
        self.ticker = ticker

    def download_data(self):
        df = yf.download(
            tickers=self.ticker,
            period=self.config.period,
            interval=self.config.interval,
            auto_adjust=True
        )
        if df.empty:
            print(f"No data for {self.ticker}")
            return df
        
        df = df.reset_index()
        df['Ticker'] = self.ticker
        df.columns = df.columns.droplevel(level=1)
        df = df[['Ticker', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']]

        df.to_csv(self.config.raw_data_file, index=False)
        print(f"Data saved to {self.config.raw_data_file}")
        return df

In [ ]:
#src/finance_ml/pipeline/stage_01_data_ingestion.py

from src.finance_ml.config.configuration import ConfigurationManager
from src.finance_ml.components.data_ingestion import DataIngestion
from src.finance_ml import logger

STAGE_NAME = "Data Ingestion stage"

class DataIngestionTrainingPipeline:
    def __init__(self, ticker: str):
        self.ticker = ticker

    def main(self):
        config = ConfigurationManager()
        data_config = config.get_data_ingestion_config()
        data_ingestor = DataIngestion(config=data_config, ticker=self.ticker)
        data_ingestor.download_data()
